In [46]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
import statsmodels.api as sm
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import metrics
import matplotlib.pyplot as plt  

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from pandas_profiling import ProfileReport

from copy import copy, deepcopy
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb 





In [47]:
train_orig = pd.read_csv( "train_cleaned.csv" )
train_orig=pd.DataFrame(train_orig)
Y_train=deepcopy(train_orig["SalePrice"])
train=deepcopy(train_orig.drop(['SalePrice'],axis=1))
print("training data size = ", train.shape)
test_orig = pd.read_csv( "test_cleaned.csv" )
test_orig=pd.DataFrame(test_orig)
print("test data size = ", test_orig.shape)

training data size =  (1456, 131)
test data size =  (1459, 131)


In [48]:
#encode
def encodeCat(categoricalData):
    encodedCategoricalData=pd.get_dummies(deepcopy(categoricalData),columns=categoricalData.columns)
    return encodedCategoricalData
def processData(train):
    categoricalData=deepcopy(train.select_dtypes(include='object'))    
    nonCategoricalData=deepcopy(train.select_dtypes(exclude='object'))
    encodedCategoricalData=encodeCat(deepcopy(categoricalData))
    #normalized=normalize(nonCategoricalData, norm='l2', axis=0, copy=True, return_norm=False)
    #nonCategoricalData=pd.DataFrame(normalized,index=nonCategoricalData.index,columns=nonCategoricalData.columns)
    #nonCategoricalData=boxcoxTransform(deepcopy(nonCategoricalData))
    nonCategoricalData=pd.concat([nonCategoricalData,encodedCategoricalData],axis=1)
    
    return nonCategoricalData
def sameSchema(X_train,X_test_orig):    
    #make sure the columns are the same and in same order
    for col in X_train.columns:
        if col not in X_test_orig.columns:
            X_test_orig[col]=np.zeros(X_test_orig.shape[0])
    X_test_orig=X_test_orig[X_train.columns]
    return X_test_orig
#combine both data sets, process and then split again
train_size=len(train)
data=deepcopy(train)
data=data.append(test_orig,sort=False)
processed = processData(deepcopy(data))
#processed=remove_corr(deepcopy(processed))
X_train_processed=deepcopy(processed[:train_size])
test_orig_processed=deepcopy(processed[train_size:len(data)])
test_orig_processed = sameSchema(deepcopy(X_train_processed),deepcopy(test_orig_processed))
#trasnform Y
Y_train_processed=deepcopy(Y_train)
#Y_train_processed = np.log(Y_train)
def featureSelector(X,Y,test,N=81):
    model=xgb.XGBRegressor()
    #model=BayesianRidge()
    model.fit(X,np.log(Y))
    imp = dict(zip(X.columns,model.feature_importances_))
    #imp = dict(zip(X_train_.columns,model.coef_))

    impsorted={k:v for k,v in sorted(imp.items(),key=lambda item:item[1],reverse=True)[:N]}
    newcols=impsorted.keys()
    return X[newcols],test[newcols]
print("X_train_ shape = ", X_train_processed.shape)



X_train_ shape =  (1456, 269)


In [49]:
def printErrors(Y_train,Y_pred):
    #print('Root Mean Squared Log Error:', round(np.sqrt(metrics.mean_squared_error(np.log(Y_train), np.log(Y_pred))),5))
    #print('Mean error:',round(np.sqrt(metrics.mean_squared_error((Y_train), (Y_pred)))))
    return np.sqrt(metrics.mean_squared_error(np.log(Y_train), np.log(Y_pred)))
def saveOutput(y_pred):
    y_pred_df=pd.DataFrame()
    y_pred_df["Id"]=range(1461,1461+len(y_pred),1)
    y_pred_df["SalePrice"]=y_pred
    file = "/Users/shaan/Downloads/DataSet/my_submission.csv"
    y_pred_df.to_csv(file,index=False)
    print("saved to file :"+file)

In [50]:
#model9
from sklearn.linear_model import BayesianRidge 
def bayesian(X_train_, X_test_, Y_train_, Y_test_,test_orig_processed):
    Y_train_exp=np.log(Y_train_)
    Y_test_exp=np.log(Y_test_)
    
    regressor = BayesianRidge(tol=1e-5,n_iter=1000,copy_X=True,fit_intercept=True)
    regressor.fit(X_train_, Y_train_exp)
    #print("------bayes---------")
    #print("Train Errors-")
    Y_pred=regressor.predict(X_train_)
    Y_pred_train=np.exp(Y_pred)
    trainerror=printErrors(Y_train_,Y_pred_train)
    #print("Test Errors-")
    Y_pred=regressor.predict(X_test_)
    Y_pred=np.exp(Y_pred)
    Y_pred_test=np.maximum(Y_pred,1)#TODO check why this is happening 
    testerror=printErrors(Y_test_,Y_pred_test)

    return regressor, trainerror, testerror

In [51]:
def runselectedModel(model,X_train_processed,Y_train_processed,test_orig_processed):
    Y_train=np.log(Y_train_processed)
    model.fit(X_train_processed,Y_train)
    
    Y_pred=model.predict(X_train_processed)
    Y_pred=np.exp(Y_pred)
    Y_pred_test=np.maximum(Y_pred,1)#TODO check why this is happening 
    
    Y_pred_dev=model.predict(test_orig_processed)
    Y_pred_dev=np.exp(Y_pred_dev)
    actual = pd.read_csv( "/Users/shaan/Downloads/DataSet/full-score.csv" )
    actual=pd.DataFrame(actual)
    #print("Actual Errors-")
    finalerror=printErrors(actual['SalePrice'],Y_pred_dev)
    return finalerror

In [52]:
def features(X_train_,bayes,j):
    #imp = dict(zip(X_train_.columns,xgmodel.feature_importances_))
    imp = dict(zip(X_train_.columns,bayes.coef_))
    impsorted={k:v for k,v in sorted(imp.items(),key=lambda item:item[1],reverse=True)[:j]}
    newcols=impsorted.keys()
    return newcols

In [100]:
Nfeatures=X_train_processed.shape[1]
errorDF=pd.DataFrame(columns=['MainLoop','InsideLoop','TrainError','TestError','AcutalError',\
                              'TrainErrorInside','TestErrorInside','AcutalErrorInside'])
for i in range(100,Nfeatures+1):
    X_tr,X_te=featureSelector(deepcopy(X_train_processed),deepcopy(Y_train_processed),\
                              deepcopy(test_orig_processed),i)
    X_train_, X_test_, Y_train_, Y_test_ = train_test_split(deepcopy(X_tr),\
                                                        deepcopy(Y_train_processed),\
                                                        test_size=0.2, random_state=0)
    model,trainerror, testerror = bayesian(deepcopy(X_train_), deepcopy(X_test_),\
                                  deepcopy(Y_train_), deepcopy(Y_test_),deepcopy(X_te))
    finalerror=runselectedModel(model,deepcopy(X_tr),deepcopy(Y_train_processed),\
                                deepcopy(X_te))
    for j in range(50,i+1):
        print(i,":",j)
        newcols=features(X_train_,model,j)
        model2,trainerror2, testerror2 = bayesian(deepcopy(X_train_[newcols]), deepcopy(X_test_[newcols]),\
                                                           deepcopy(Y_train_), deepcopy(Y_test_),\
                                                           deepcopy(X_te[newcols]))
        finalerror2 = runselectedModel(model2,deepcopy(X_tr[newcols]),deepcopy(Y_train_processed),\
                 deepcopy(X_te[newcols]))
        
        errorDF.loc[len(errorDF)]=[i,j,trainerror, testerror,finalerror,trainerror2, testerror2,finalerror2]

100 : 50
100 : 51
100 : 52
100 : 53
100 : 54
100 : 55
100 : 56
100 : 57
100 : 58
100 : 59
100 : 60
100 : 61
100 : 62
100 : 63
100 : 64
100 : 65
100 : 66
100 : 67
100 : 68
100 : 69
100 : 70
100 : 71
100 : 72
100 : 73
100 : 74
100 : 75
100 : 76
100 : 77
100 : 78
100 : 79
100 : 80
100 : 81
100 : 82
100 : 83
100 : 84
100 : 85
100 : 86
100 : 87
100 : 88
100 : 89
100 : 90
100 : 91
100 : 92
100 : 93
100 : 94
100 : 95
100 : 96
100 : 97
100 : 98
100 : 99
100 : 100
101 : 50
101 : 51
101 : 52
101 : 53
101 : 54
101 : 55
101 : 56
101 : 57
101 : 58
101 : 59
101 : 60
101 : 61
101 : 62
101 : 63
101 : 64
101 : 65
101 : 66
101 : 67
101 : 68
101 : 69
101 : 70
101 : 71
101 : 72
101 : 73
101 : 74
101 : 75
101 : 76
101 : 77
101 : 78
101 : 79
101 : 80
101 : 81
101 : 82
101 : 83
101 : 84
101 : 85
101 : 86
101 : 87
101 : 88
101 : 89
101 : 90
101 : 91
101 : 92
101 : 93
101 : 94
101 : 95
101 : 96
101 : 97
101 : 98
101 : 99
101 : 100
101 : 101
102 : 50
102 : 51
102 : 52
102 : 53
102 : 54
102 : 55
102 : 56
102 : 5

115 : 80
115 : 81
115 : 82
115 : 83
115 : 84
115 : 85
115 : 86
115 : 87
115 : 88
115 : 89
115 : 90
115 : 91
115 : 92
115 : 93
115 : 94
115 : 95
115 : 96
115 : 97
115 : 98
115 : 99
115 : 100
115 : 101
115 : 102
115 : 103
115 : 104
115 : 105
115 : 106
115 : 107
115 : 108
115 : 109
115 : 110
115 : 111
115 : 112
115 : 113
115 : 114
115 : 115
116 : 50
116 : 51
116 : 52
116 : 53
116 : 54
116 : 55
116 : 56
116 : 57
116 : 58
116 : 59
116 : 60
116 : 61
116 : 62
116 : 63
116 : 64
116 : 65
116 : 66
116 : 67
116 : 68
116 : 69
116 : 70
116 : 71
116 : 72
116 : 73
116 : 74
116 : 75
116 : 76
116 : 77
116 : 78
116 : 79
116 : 80
116 : 81
116 : 82
116 : 83
116 : 84
116 : 85
116 : 86
116 : 87
116 : 88
116 : 89
116 : 90
116 : 91
116 : 92
116 : 93
116 : 94
116 : 95
116 : 96
116 : 97
116 : 98
116 : 99
116 : 100
116 : 101
116 : 102
116 : 103
116 : 104
116 : 105
116 : 106
116 : 107
116 : 108
116 : 109
116 : 110
116 : 111
116 : 112
116 : 113
116 : 114
116 : 115
116 : 116
117 : 50
117 : 51
117 : 52
117 : 53
117 

127 : 105
127 : 106
127 : 107
127 : 108
127 : 109
127 : 110
127 : 111
127 : 112
127 : 113
127 : 114
127 : 115
127 : 116
127 : 117
127 : 118
127 : 119
127 : 120
127 : 121
127 : 122
127 : 123
127 : 124
127 : 125
127 : 126
127 : 127
128 : 50
128 : 51
128 : 52
128 : 53
128 : 54
128 : 55
128 : 56
128 : 57
128 : 58
128 : 59
128 : 60
128 : 61
128 : 62
128 : 63
128 : 64
128 : 65
128 : 66
128 : 67
128 : 68
128 : 69
128 : 70
128 : 71
128 : 72
128 : 73
128 : 74
128 : 75
128 : 76
128 : 77
128 : 78
128 : 79
128 : 80
128 : 81
128 : 82
128 : 83
128 : 84
128 : 85
128 : 86
128 : 87
128 : 88
128 : 89
128 : 90
128 : 91
128 : 92
128 : 93
128 : 94
128 : 95
128 : 96
128 : 97
128 : 98
128 : 99
128 : 100
128 : 101
128 : 102
128 : 103
128 : 104
128 : 105
128 : 106
128 : 107
128 : 108
128 : 109
128 : 110
128 : 111
128 : 112
128 : 113
128 : 114
128 : 115
128 : 116
128 : 117
128 : 118
128 : 119
128 : 120
128 : 121
128 : 122
128 : 123
128 : 124
128 : 125
128 : 126
128 : 127
128 : 128
129 : 50
129 : 51
129 : 52
129

138 : 66
138 : 67
138 : 68
138 : 69
138 : 70
138 : 71
138 : 72
138 : 73
138 : 74
138 : 75
138 : 76
138 : 77
138 : 78
138 : 79
138 : 80
138 : 81
138 : 82
138 : 83
138 : 84
138 : 85
138 : 86
138 : 87
138 : 88
138 : 89
138 : 90
138 : 91
138 : 92
138 : 93
138 : 94
138 : 95
138 : 96
138 : 97
138 : 98
138 : 99
138 : 100
138 : 101
138 : 102
138 : 103
138 : 104
138 : 105
138 : 106
138 : 107
138 : 108
138 : 109
138 : 110
138 : 111
138 : 112
138 : 113
138 : 114
138 : 115
138 : 116
138 : 117
138 : 118
138 : 119
138 : 120
138 : 121
138 : 122
138 : 123
138 : 124
138 : 125
138 : 126
138 : 127
138 : 128
138 : 129
138 : 130
138 : 131
138 : 132
138 : 133
138 : 134
138 : 135
138 : 136
138 : 137
138 : 138
139 : 50
139 : 51
139 : 52
139 : 53
139 : 54
139 : 55
139 : 56
139 : 57
139 : 58
139 : 59
139 : 60
139 : 61
139 : 62
139 : 63
139 : 64
139 : 65
139 : 66
139 : 67
139 : 68
139 : 69
139 : 70
139 : 71
139 : 72
139 : 73
139 : 74
139 : 75
139 : 76
139 : 77
139 : 78
139 : 79
139 : 80
139 : 81
139 : 82
139 : 8

147 : 97
147 : 98
147 : 99
147 : 100
147 : 101
147 : 102
147 : 103
147 : 104
147 : 105
147 : 106
147 : 107
147 : 108
147 : 109
147 : 110
147 : 111
147 : 112
147 : 113
147 : 114
147 : 115
147 : 116
147 : 117
147 : 118
147 : 119
147 : 120
147 : 121
147 : 122
147 : 123
147 : 124
147 : 125
147 : 126
147 : 127
147 : 128
147 : 129
147 : 130
147 : 131
147 : 132
147 : 133
147 : 134
147 : 135
147 : 136
147 : 137
147 : 138
147 : 139
147 : 140
147 : 141
147 : 142
147 : 143
147 : 144
147 : 145
147 : 146
147 : 147
148 : 50
148 : 51
148 : 52
148 : 53
148 : 54
148 : 55
148 : 56
148 : 57
148 : 58
148 : 59
148 : 60
148 : 61
148 : 62
148 : 63
148 : 64
148 : 65
148 : 66
148 : 67
148 : 68
148 : 69
148 : 70
148 : 71
148 : 72
148 : 73
148 : 74
148 : 75
148 : 76
148 : 77
148 : 78
148 : 79
148 : 80
148 : 81
148 : 82
148 : 83
148 : 84
148 : 85
148 : 86
148 : 87
148 : 88
148 : 89
148 : 90
148 : 91
148 : 92
148 : 93
148 : 94
148 : 95
148 : 96
148 : 97
148 : 98
148 : 99
148 : 100
148 : 101
148 : 102
148 : 103
148

...

254 : 75
254 : 76
254 : 77
254 : 78
254 : 79
254 : 80
254 : 81
254 : 82
254 : 83
254 : 84
254 : 85
254 : 86
254 : 87
254 : 88
254 : 89
254 : 90
254 : 91
254 : 92
254 : 93
254 : 94
254 : 95
254 : 96
254 : 97
254 : 98
254 : 99
254 : 100
254 : 101
254 : 102
254 : 103
254 : 104
254 : 105
254 : 106
254 : 107
254 : 108
254 : 109
254 : 110
254 : 111
254 : 112
254 : 113
254 : 114
254 : 115
254 : 116
254 : 117
254 : 118
254 : 119
254 : 120
254 : 121
254 : 122
254 : 123
254 : 124
254 : 125
254 : 126
254 : 127
254 : 128
254 : 129
254 : 130
254 : 131
254 : 132
254 : 133
254 : 134
254 : 135
254 : 136
254 : 137
254 : 138
254 : 139
254 : 140
254 : 141
254 : 142
254 : 143
254 : 144
254 : 145
254 : 146
254 : 147
254 : 148
254 : 149
254 : 150
254 : 151
254 : 152
254 : 153
254 : 154
254 : 155
254 : 156
254 : 157
254 : 158
254 : 159
254 : 160
254 : 161
254 : 162
254 : 163
254 : 164
254 : 165
254 : 166
254 : 167
254 : 168
254 : 169
254 : 170
254 : 171
254 : 172
254 : 173
254 : 174
254 : 175
254 : 176
254 :

258 : 93
258 : 94
258 : 95
258 : 96
258 : 97
258 : 98
258 : 99
258 : 100
258 : 101
258 : 102
258 : 103
258 : 104
258 : 105
258 : 106
258 : 107
258 : 108
258 : 109
258 : 110
258 : 111
258 : 112
258 : 113
258 : 114
258 : 115
258 : 116
258 : 117
258 : 118
258 : 119
258 : 120
258 : 121
258 : 122
258 : 123
258 : 124
258 : 125
258 : 126
258 : 127
258 : 128
258 : 129
258 : 130
258 : 131
258 : 132
258 : 133
258 : 134
258 : 135
258 : 136
258 : 137
258 : 138
258 : 139
258 : 140
258 : 141
258 : 142
258 : 143
258 : 144
258 : 145
258 : 146
258 : 147
258 : 148
258 : 149
258 : 150
258 : 151
258 : 152
258 : 153
258 : 154
258 : 155
258 : 156
258 : 157
258 : 158
258 : 159
258 : 160
258 : 161
258 : 162
258 : 163
258 : 164
258 : 165
258 : 166
258 : 167
258 : 168
258 : 169
258 : 170
258 : 171
258 : 172
258 : 173
258 : 174
258 : 175
258 : 176
258 : 177
258 : 178
258 : 179
258 : 180
258 : 181
258 : 182
258 : 183
258 : 184
258 : 185
258 : 186
258 : 187
258 : 188
258 : 189
258 : 190
258 : 191
258 : 192
258 : 1

262 : 93
262 : 94
262 : 95
262 : 96
262 : 97
262 : 98
262 : 99
262 : 100
262 : 101
262 : 102
262 : 103
262 : 104
262 : 105
262 : 106
262 : 107
262 : 108
262 : 109
262 : 110
262 : 111
262 : 112
262 : 113
262 : 114
262 : 115
262 : 116
262 : 117
262 : 118
262 : 119
262 : 120
262 : 121
262 : 122
262 : 123
262 : 124
262 : 125
262 : 126
262 : 127
262 : 128
262 : 129
262 : 130
262 : 131
262 : 132
262 : 133
262 : 134
262 : 135
262 : 136
262 : 137
262 : 138
262 : 139
262 : 140
262 : 141
262 : 142
262 : 143
262 : 144
262 : 145
262 : 146
262 : 147
262 : 148
262 : 149
262 : 150
262 : 151
262 : 152
262 : 153
262 : 154
262 : 155
262 : 156
262 : 157
262 : 158
262 : 159
262 : 160
262 : 161
262 : 162
262 : 163
262 : 164
262 : 165
262 : 166
262 : 167
262 : 168
262 : 169
262 : 170
262 : 171
262 : 172
262 : 173
262 : 174
262 : 175
262 : 176
262 : 177
262 : 178
262 : 179
262 : 180
262 : 181
262 : 182
262 : 183
262 : 184
262 : 185
262 : 186
262 : 187
262 : 188
262 : 189
262 : 190
262 : 191
262 : 192
262 : 1

266 : 73
266 : 74
266 : 75
266 : 76
266 : 77
266 : 78
266 : 79
266 : 80
266 : 81
266 : 82
266 : 83
266 : 84
266 : 85
266 : 86
266 : 87
266 : 88
266 : 89
266 : 90
266 : 91
266 : 92
266 : 93
266 : 94
266 : 95
266 : 96
266 : 97
266 : 98
266 : 99
266 : 100
266 : 101
266 : 102
266 : 103
266 : 104
266 : 105
266 : 106
266 : 107
266 : 108
266 : 109
266 : 110
266 : 111
266 : 112
266 : 113
266 : 114
266 : 115
266 : 116
266 : 117
266 : 118
266 : 119
266 : 120
266 : 121
266 : 122
266 : 123
266 : 124
266 : 125
266 : 126
266 : 127
266 : 128
266 : 129
266 : 130
266 : 131
266 : 132
266 : 133
266 : 134
266 : 135
266 : 136
266 : 137
266 : 138
266 : 139
266 : 140
266 : 141
266 : 142
266 : 143
266 : 144
266 : 145
266 : 146
266 : 147
266 : 148
266 : 149
266 : 150
266 : 151
266 : 152
266 : 153
266 : 154
266 : 155
266 : 156
266 : 157
266 : 158
266 : 159
266 : 160
266 : 161
266 : 162
266 : 163
266 : 164
266 : 165
266 : 166
266 : 167
266 : 168
266 : 169
266 : 170
266 : 171
266 : 172
266 : 173
266 : 174
266 : 1

269 : 256
269 : 257
269 : 258
269 : 259
269 : 260
269 : 261
269 : 262
269 : 263
269 : 264
269 : 265
269 : 266
269 : 267
269 : 268
269 : 269


In [101]:
print("min error Val:",min(errorDF['AcutalErrorInside']))
a=np.argmin(np.array(errorDF['AcutalErrorInside']))
print("min error config index:",a)
print(errorDF.loc[a])
errorDF.sort_values('AcutalErrorInside')

min error Val: 0.12384810207291275
min error config index: 16730
MainLoop             239.000000
InsideLoop           100.000000
TrainError             0.098337
TestError              0.102667
AcutalError            0.126406
TrainErrorInside       0.104037
TestErrorInside        0.103190
AcutalErrorInside      0.123848
Name: 16730, dtype: float64


,MainLoop,InsideLoop,TrainError,TestError,AcutalError,TrainErrorInside,TestErrorInside,AcutalErrorInside
16730,239.0,100.0,0.098337,0.102667,0.126406,0.104037,0.103190,0.123848
17110,241.0,99.0,0.098350,0.102678,0.126409,0.104007,0.103150,0.123854
16919,240.0,99.0,0.098346,0.102676,0.126408,0.104007,0.103150,0.123854
17691,244.0,101.0,0.098376,0.102763,0.126493,0.104048,0.103137,0.123889
17496,243.0,100.0,0.098384,0.102714,0.126438,0.104018,0.103096,0.123896
...,...,...,...,...,...,...,...,...
21730,264.0,50.0,0.098243,0.102697,0.126461,0.117916,0.119624,0.137192
21091,261.0,50.0,0.098248,0.102774,0.126492,0.117916,0.119624,0.137192
20880,260.0,50.0,0.098243,0.102773,0.126499,0.117916,0.119624,0.137192
21516,263.0,50.0,0.098245,0.102743,0.126486,0.117916,0.119624,0.137192


In [76]:
'''
def runandsave(model,X_train_processed,Y_train_processed,test_orig_processed):
    Y_train=np.log(Y_train_processed)
    model.fit(X_train_processed,Y_train)
    
    Y_pred=model.predict(X_train_processed)
    Y_pred=np.exp(Y_pred)
    Y_pred_test=np.maximum(Y_pred,1)#TODO check why this is happening 
    
    Y_pred_dev=model.predict(test_orig_processed)
    Y_pred_dev=np.exp(Y_pred_dev)
    actual = pd.read_csv( "/Users/shaan/Downloads/DataSet/full-score.csv" )
    actual=pd.DataFrame(actual)
    #print("Actual Errors-")
    finalerror=printErrors(actual['SalePrice'],Y_pred_dev)
    
    print("Saving output for upload")
    saveOutput(Y_pred_dev)
    return finalerror
'''

In [106]:
'''
#rerun with the config
i=241
j=99
X_tr,X_te=featureSelector(deepcopy(X_train_processed),deepcopy(Y_train_processed),\
                              deepcopy(test_orig_processed),i)
X_train_, X_test_, Y_train_, Y_test_ = train_test_split(deepcopy(X_tr),\
                                                    deepcopy(Y_train_processed),\
                                                    test_size=0.2, random_state=0)
model,trainerror, testerror = bayesian(deepcopy(X_train_), deepcopy(X_test_),\
                              deepcopy(Y_train_), deepcopy(Y_test_),deepcopy(X_te))
finalerror=runselectedModel(model,deepcopy(X_tr),deepcopy(Y_train_processed),\
                            deepcopy(X_te))
newcols=features(X_train_,model,j)
model2,trainerror2, testerror2 = bayesian(deepcopy(X_train_[newcols]), deepcopy(X_test_[newcols]),\
                                                   deepcopy(Y_train_), deepcopy(Y_test_),\
                                                   deepcopy(X_te[newcols]))
finalerror2 = runandsave(model2,deepcopy(X_tr[newcols]),deepcopy(Y_train_processed),\
         deepcopy(X_te[newcols]))
'''

Saving output for upload
saved to file :/Users/shaan/Downloads/DataSet/my_submission.csv
